# Download dataset via api do Kaggle

In [5]:
from google.colab import files
files.upload()
 
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
 
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
 
!unzip \*.zip  && rm *.zip

Saving kaggle.json to kaggle.json
 19% 5.00M/25.7M [00:00<00:00, 38.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 101MB/s] 
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


# Importação das bibliotecas

In [6]:
import pandas as pd
import re
import spacy
import numpy as np
import sklearn
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

# Função para remover tags html, retirar apostrofos e outros caracteres

In [7]:
def cleaner(x):
  x = x.map(lambda z: z.lower())
  x = x.map(lambda z: re.sub(r"<.*?>", '', z))
  x = x.map(lambda z: z.replace("'", ''))
  x = x.map(lambda z: re.sub(r"[^a-zA-Z]+", ' ', z))
  return x

Leitura do dataset em CSV

In [8]:
ds = pd.read_csv('./IMDB Dataset.csv')
ds.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Limpeza do dataset

In [9]:
ds['review'] = cleaner(ds['review'])

In [10]:
ds.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


In [11]:
print(ds['review'][0])

one of the other reviewers has mentioned that after watching just oz episode youll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare forget pretty pict

# Utilizando modelo do Spacy para obter o vetor relativo as sentenças

In [12]:
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm')

In [14]:
x = 'time and relative dimensions in space'
doc = nlp(x)

print(doc.vector.shape)


(96,)


In [15]:
def get_vector(x):
  x_pross = nlp(x)
  doc = x_pross.vector
  return doc


In [16]:

ds['vec'] = ds['review'].apply(lambda x: get_vector(x))

In [17]:
ds.tail()

,review,sentiment,vec
49995,i thought this movie did a down right good job...,positive,"[0.13268553, 1.0801342, 0.23692743, -1.0778698..."
49996,bad plot bad dialogue bad acting idiotic direc...,negative,"[0.15337458, 0.1500213, 0.3735996, 0.000313492..."
49997,i am a catholic taught in parochial elementary...,negative,"[0.17584437, 0.3238792, 0.29797003, -0.3708062..."
49998,im going to have to disagree with the previous...,negative,"[0.04304632, 0.43463242, 0.069901794, -0.05309..."
49999,no one expects the star trek movies to be high...,negative,"[-0.341987, 0.3608256, 0.2637007, -0.5481177, ..."


###Conversão dos vetores para matriz numpy

In [22]:
X = ds['vec'].to_numpy()
X = X.reshape(-1, 1)
X.shape

(50000, 1)

### Construção do vetores de treino e de teste

In [23]:
X = np.concatenate(np.concatenate(X)).reshape(-1, 96)
y = ds['sentiment']

In [24]:
X.shape

(50000, 96)

In [25]:
X_train, X_test, y_tain, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train.shape, X_test.shape

((40000, 96), (10000, 96))

# Construção e treinamento do modelo

In [37]:
grid_params = {
    'solver': ['sag', 'saga', 'lbfgs'],
    'penalty': ['none', 'l2']
}
model = GridSearchCV(LogisticRegression(n_jobs=4), grid_params, verbose=1)
model.fit(X_train, y_tain)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=4, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'penalty': ['none', 'l2'],
                         'solver': ['sag', 'saga', 'lbfgs']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

# Teste no vetor de teste

In [38]:
pred = model.predict(X_test)

In [39]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.70      0.71      0.71      5000
    positive       0.71      0.70      0.70      5000

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000



# Salvando modelo com Pickle

In [41]:
import pickle

with open('word2vec_sentiment_an.pkl', 'wb') as file:
  pickle.dump(model, file)